<a href="https://colab.research.google.com/github/KirtiNayak11/ML/blob/main/FrenchToEnglishTranslator_SeqToSeq_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.utils import pad_sequences

In [ ]:
english_sentences

array([[12, 13, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [15, 16, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [18, 19, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0]], dtype=int32)

In [ ]:
french_sentences

array([[ 2,  3,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 5,  6,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 8,  9, 10, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0]], dtype=int32)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,SimpleRNN
from keras import optimizers
import numpy as np

In [ ]:
# Define constants
latent_dim = 256

# Define the encoder
encoder_input = Input(shape=(max_seq_len,))
encoder_embedding = tf.keras.layers.Embedding(num_french_tokens, latent_dim)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding(encoder_input))
encoder_states = [state_h, state_c]

# Define the decoder
decoder_input = Input(shape=(max_seq_len,))
decoder_embedding = tf.keras.layers.Embedding(num_english_tokens, latent_dim)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding(decoder_input), initial_state=encoder_states)
decoder_dense = Dense(num_english_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the Seq2Seq model
model = Model([encoder_input, decoder_input], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([french_indices_padded, english_indices_padded], np.expand_dims(english_indices_padded, -1), epochs=100)

In [ ]:
# Inference
def translate_sentence(sentence, token_to_index, model):
    sentence_indices = sentences_to_indices([sentence], token_to_index)
    sentence_indices_padded = tf.keras.preprocessing.sequence.pad_sequences(sentence_indices, maxlen=max_seq_len, padding='post')
    predicted_indices = model.predict([sentence_indices_padded, sentence_indices_padded])
    predicted_sentence = " ".join([list(english_token_to_index.keys())[np.argmax(pred)] for pred in predicted_indices[0]])
    return predicted_sentence

# Test translation
input_sentence = "nous aimons la musique"
translated_sentence = translate_sentence(input_sentence, french_token_to_index, model)
print("Input:", input_sentence)
print("Translated:", translated_sentence)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Sample parallel data (French to English)
french_sentences = ["je suis content", "elle parle francais", "nous aimons la musique"]
english_sentences = ["I am happy", "she speaks French", "we love music"]

# Create vocabulary for French and English
french_vocab = set(" ".join(french_sentences).split())
english_vocab = set(" ".join(english_sentences).split())
num_french_tokens = len(french_vocab)
num_english_tokens = len(english_vocab)

# Create token-to-index dictionaries for both languages
french_token_to_index = {token: index for index, token in enumerate(french_vocab)}
english_token_to_index = {token: index for index, token in enumerate(english_vocab)}

# Convert sentences to token indices
def sentences_to_indices(sentences, token_to_index):
    return [[token_to_index[token] for token in sentence.split()] for sentence in sentences]

french_indices = sentences_to_indices(french_sentences, french_token_to_index)
english_indices = sentences_to_indices(english_sentences, english_token_to_index)

# Padding for sequences
max_seq_len = max(max(len(seq) for seq in french_indices), max(len(seq) for seq in english_indices))
french_indices_padded = tf.keras.preprocessing.sequence.pad_sequences(french_indices, maxlen=max_seq_len, padding='post')
english_indices_padded = tf.keras.preprocessing.sequence.pad_sequences(english_indices, maxlen=max_seq_len, padding='post')

# Define constants
latent_dim = 256

# Define the encoder
encoder_input = Input(shape=(max_seq_len,))
encoder_embedding = tf.keras.layers.Embedding(num_french_tokens, latent_dim)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding(encoder_input))
encoder_states = [state_h, state_c]

# Define the decoder
decoder_input = Input(shape=(max_seq_len,))
decoder_embedding = tf.keras.layers.Embedding(num_english_tokens, latent_dim)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding(decoder_input), initial_state=encoder_states)
decoder_dense = Dense(num_english_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the Seq2Seq model
model = Model([encoder_input, decoder_input], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([french_indices_padded, english_indices_padded], np.expand_dims(english_indices_padded, -1), epochs=100)

# Inference
def translate_sentence(sentence, token_to_index, model):
    sentence_indices = sentences_to_indices([sentence], token_to_index)
    sentence_indices_padded = tf.keras.preprocessing.sequence.pad_sequences(sentence_indices, maxlen=max_seq_len, padding='post')
    predicted_indices = model.predict([sentence_indices_padded, sentence_indices_padded])
    predicted_sentence = " ".join([list(english_token_to_index.keys())[np.argmax(pred)] for pred in predicted_indices[0]])
    return predicted_sentence

# Test translation
input_sentence = "nous aimons la musique"
translated_sentence = translate_sentence(input_sentence, french_token_to_index, model)
print("Input:", input_sentence)
print("Translated:", translated_sentence)


Epoch 1/100
1/1 [==============================] - 6s 6s/step - loss: 2.2005 - accuracy: 0.0833
Epoch 2/100
1/1 [==============================] - 0s 50ms/step - loss: 2.1656 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 46ms/step - loss: 2.1299 - accuracy: 0.4167
Epoch 4/100
1/1 [==============================] - 0s 47ms/step - loss: 2.0916 - accuracy: 0.3333
Epoch 5/100
1/1 [==============================] - 0s 44ms/step - loss: 2.0490 - accuracy: 0.3333
Epoch 6/100
1/1 [==============================] - 0s 46ms/step - loss: 2.0001 - accuracy: 0.3333
Epoch 7/100
1/1 [==============================] - 0s 48ms/step - loss: 1.9432 - accuracy: 0.3333
Epoch 8/100
1/1 [==============================] - 0s 51ms/step - loss: 1.8764 - accuracy: 0.3333
Epoch 9/100
1/1 [==============================] - 0s 48ms/step - loss: 1.7986 - accuracy: 0.3333
Epoch 10/100
1/1 [==============================] - 0s 41ms/step - loss: 1.7108 - accuracy: 0.3333
Epoch 11/100
1/1 [===